<div class="alert alert-block alert-warning">
    <b>WARNING! </b> This notebook will overwrite the file saved in the HDFS. Since we already generated them for you, there is no need to run this notebook unless you want to verify our solution from scratch. <br>
    To run the notebook without the risk of overwriting, we set a flag called "overwrite" to false. If you want to overwrite the files in the HDFS set it to true
</div>

## Spark Loading

In [1]:
%load_ext sparkmagic.magics

In [2]:
import os
from IPython import get_ipython
username = os.environ['RENKU_USERNAME']
server = "http://iccluster029.iccluster.epfl.ch:8998"

# set the application name as "<your_gaspar_id>-homework3"
get_ipython().run_cell_magic(
    'spark',
    line='config', 
    cell="""{{ "name": "{0}-homework3", "executorMemory": "4G", "executorCores": 4, "numExecutors": 10, "driverMemory": "4G"}}""".format(username)
)

In [3]:
get_ipython().run_line_magic(
    "spark", "add -s {0}-homework3 -l python -u {1} -k".format(username, server)
)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
9298,application_1652960972356_5105,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


In [4]:
%%spark
print('We are using Spark %s' % spark.version)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

We are using Spark 2.3.2.3.1.4.0-315

In [5]:
%%spark
overwrite = False

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Stops 15km from Zürich HB (8503000) (lat, lon) = (47.378177, 8.540192)

In [6]:
%%spark
all_stops = spark.read.csv('/data/sbb/csv/allstops/stop_locations.csv')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
%%spark
all_stops = all_stops.withColumnRenamed('_c0','stop_id')
all_stops = all_stops.withColumnRenamed('_c1','stop_name')
all_stops = all_stops.withColumnRenamed('_c2','lat')
all_stops = all_stops.withColumnRenamed('_c3','lon')
all_stops = all_stops.drop('_c4','_c5')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
%%spark
import pyspark.sql.functions as F
zur_hp_lat = 47.378177
zur_hp_lon = 8.540192

all_stops_distance = all_stops.withColumn("a", (
        F.pow(F.sin(F.radians(zur_hp_lat - F.col("lat")) / 2), 2) +
        F.cos(F.radians(F.col("lat"))) * F.cos(F.radians(F.lit(zur_hp_lat))) *
        F.pow(F.sin(F.radians(zur_hp_lon - F.col("lon")) / 2), 2)
    )).withColumn("distance", F.atan2(F.sqrt(F.col("a")), F.sqrt(-F.col("a") + 1)) * 12742000)

all_stops_distance.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+----------------+----------------+--------------------+------------------+
|stop_id|           stop_name|             lat|             lon|                   a|          distance|
+-------+--------------------+----------------+----------------+--------------------+------------------+
|1100006|Zell (Wiesental),...|47.7046317515335|7.84777215468897|2.475398503604094...|  63396.0142787592|
|1100008|Zell (Wiesental),...|47.7100842702352|7.85964788274668|2.445985737384344...| 63018.24892966985|
|1100009|Zell (Wiesental),...|47.7131911044794|7.86290876722849|2.446306465224431...| 63022.38043559904|
|1100010|           Atzenbach|47.7146175266411| 7.8723500608659|2.409494241610368E-5|62546.396144882805|
|1100011|     Mambach, Brücke|47.7282088873189| 7.8774704579861|2.456510835903595E-5| 63153.68854022285|
|1100012|  Mambach, Mühlschau|47.7340818684375| 7.8813871126254|2.469960438154199...| 63326.33982721011|
|1100013|  Mambach, Silbersau|47.7395192233867|7.882231

In [9]:
%%spark
close_stops = all_stops_distance.filter(all_stops_distance['distance'] <= 15000).drop('a','distance')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
%%spark
close_stops.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2122

## Weekly timetable

Considered period: May 08-14, 2019 (then we exclude the weekends)

In [11]:
%%spark
all_trips1 = spark.read.csv('/data/sbb/csv/stop_times/2019/05/08/stop_times.txt', header=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
%%spark
#all_trips_tot = all_trips1.union(all_trips2)
all_trips_tot = all_trips1
all_trips_tot.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+------------+--------------+-------+-------------+-----------+-------------+
|            trip_id|arrival_time|departure_time|stop_id|stop_sequence|pickup_type|drop_off_type|
+-------------------+------------+--------------+-------+-------------+-----------+-------------+
|1.TA.1-84-j19-1.1.H|    06:13:00|      06:13:00|8572249|            1|          0|            0|
|1.TA.1-84-j19-1.1.H|    06:14:00|      06:14:00|8577508|            2|          0|            0|
|1.TA.1-84-j19-1.1.H|    06:15:00|      06:15:00|8581070|            3|          0|            0|
|1.TA.1-84-j19-1.1.H|    06:16:00|      06:16:00|8578360|            4|          0|            0|
|1.TA.1-84-j19-1.1.H|    06:17:00|      06:17:00|8583448|            5|          0|            0|
|1.TA.1-84-j19-1.1.H|    06:18:00|      06:19:00|8578359|            6|          0|            0|
|1.TA.1-84-j19-1.1.H|    06:24:00|      06:24:00|8578358|            7|          0|            0|
|1.TA.1-84-j19-1.1.H

In [13]:
%%spark
routes_tot = spark.read.csv('/data/sbb/csv/trips/2019/05/08/trips.txt', header=True)
routes_tot.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----------+--------------------+-------------------+---------------+------------+
|   route_id|service_id|             trip_id|      trip_headsign|trip_short_name|direction_id|
+-----------+----------+--------------------+-------------------+---------------+------------+
| 1-85-j19-1|  TA+b0001| 2.TA.1-85-j19-1.1.H|Schöftland, Bahnhof|          85003|           0|
|1-1-C-j19-1|  TA+b0001|5.TA.1-1-C-j19-1.3.R| Zofingen, Altachen|            108|           1|
|1-1-C-j19-1|  TA+b0001|7.TA.1-1-C-j19-1.3.R| Zofingen, Altachen|            112|           1|
|1-1-C-j19-1|  TA+b0001|9.TA.1-1-C-j19-1.3.R| Zofingen, Altachen|            116|           1|
|1-1-C-j19-1|  TA+b0001|11.TA.1-1-C-j19-1...| Zofingen, Altachen|            120|           1|
|1-1-C-j19-1|  TA+b0001|13.TA.1-1-C-j19-1...| Zofingen, Altachen|            124|           1|
|1-1-C-j19-1|  TA+b0001|15.TA.1-1-C-j19-1...| Zofingen, Altachen|            128|           1|
|1-1-C-j19-1|  TA+b0001|17.TA.1-1-C-j19-1...| Zofi

In [14]:
%%spark
trips_with_routes = all_trips_tot.join(routes_tot, all_trips_tot.trip_id == routes_tot.trip_id)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
%%spark
trips_with_routes = trips_with_routes.select(all_trips_tot.trip_id, 'service_id').distinct()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
%%spark
cal1 = spark.read.csv('/data/sbb/csv/calendar/2019/05/08/calendar.txt', header=True)
cal_tot = cal1.distinct()
cal_tot.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+-------+---------+--------+------+--------+------+----------+--------+
|service_id|monday|tuesday|wednesday|thursday|friday|saturday|sunday|start_date|end_date|
+----------+------+-------+---------+--------+------+--------+------+----------+--------+
|  TA+b06hv|     0|      0|        0|       0|     0|       0|     1|  20181209|20191214|
|  TA+b0036|     0|      0|        0|       0|     0|       0|     1|  20181209|20191214|
|  TA+b0ha1|     0|      0|        0|       0|     0|       1|     0|  20181209|20191214|
|  TA+b0nha|     1|      1|        1|       1|     0|       0|     1|  20181209|20191214|
|  TA+b022q|     1|      1|        1|       1|     0|       0|     0|  20181209|20191214|
|  TA+b025m|     0|      0|        0|       0|     1|       0|     0|  20181209|20191214|
|  TA+b0ms4|     0|      0|        0|       0|     0|       0|     1|  20181209|20191214|
|  TA+b0nvl|     0|      0|        0|       0|     0|       1|     1|  20181209|20191214|
|  TA+b0nz

In [17]:
%%spark
cal_filtered = cal_tot.filter((cal_tot.monday == 1) | (cal_tot.tuesday == 1) | (cal_tot.wednesday == 1) | (cal_tot.thursday == 1) | (cal_tot.friday == 1))
cal_filtered_dates = cal_filtered.filter((cal_tot.start_date <= '20190513') & (cal_tot.end_date >= '20190518'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
%%spark
service_ids = cal_filtered_dates.drop('start_date', 'end_date', 'saturday', 'sunday')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
%%spark
service_ids.select('service_id')
trips_in_period = trips_with_routes.join(service_ids, service_ids.service_id == trips_with_routes.service_id,'inner').drop(trips_with_routes.service_id).distinct()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
%%spark
final_trips = all_trips_tot.join(trips_in_period, trips_in_period.trip_id == all_trips_tot.trip_id).distinct().drop(all_trips_tot.trip_id)
final_trips.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------+------------+-------------+-----------+-------------+--------------------+----------+------+-------+---------+--------+------+
|arrival_time|departure_time|     stop_id|stop_sequence|pickup_type|drop_off_type|             trip_id|service_id|monday|tuesday|wednesday|thursday|friday|
+------------+--------------+------------+-------------+-----------+-------------+--------------------+----------+------+-------+---------+--------+------+
|    01:30:00|      01:30:00|8505000:0:11|            1|          0|            0|1.TA.12-1-A-j19-1...|  TA+b099q|     0|      1|        1|       0|     1|
|    01:35:00|      01:35:00| 8502021:0:1|            2|          0|            0|1.TA.12-1-A-j19-1...|  TA+b099q|     0|      1|        1|       0|     1|
|    01:36:00|      01:37:00| 8502028:0:1|            3|          0|            0|1.TA.12-1-A-j19-1...|  TA+b099q|     0|      1|        1|       0|     1|
|    01:39:00|      01:39:00| 8502012:0:1|            4|        

In [21]:
%%spark
all_trips_tot.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

10862563

In [22]:
%%spark
final_trips.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

7008267

### Considering working hours from 9:00 to 17:59

In [23]:
%%spark
import pyspark.sql.functions as F

final_trips = final_trips.filter(F.hour(final_trips.arrival_time).between(9,17) & F.hour(final_trips.departure_time).between(9,17))
final_trips.show(10, truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------+-----------+-------------+-----------+-------------+--------------------+----------+------+-------+---------+--------+------+
|arrival_time|departure_time|stop_id    |stop_sequence|pickup_type|drop_off_type|trip_id             |service_id|monday|tuesday|wednesday|thursday|friday|
+------------+--------------+-----------+-------------+-----------+-------------+--------------------+----------+------+-------+---------+--------+------+
|10:41:00    |10:41:00      |8503064    |1            |0          |0            |1.TA.26-18-j19-1.1.H|TA+b0b46  |1     |1      |1        |1       |1     |
|10:45:00    |10:45:00      |8503065:0:1|2            |0          |0            |1.TA.26-18-j19-1.1.H|TA+b0b46  |1     |1      |1        |1       |1     |
|10:46:00    |10:46:00      |8503074    |3            |0          |0            |1.TA.26-18-j19-1.1.H|TA+b0b46  |1     |1      |1        |1       |1     |
|10:47:00    |10:47:00      |8503068    |4            |0          |0  

In [24]:
%%spark
final_trips.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3777527

### Keeping only trip_ids that contains a stop in the area of Zurich

In [25]:
%%spark
final_close_trips_ids = final_trips.join(close_stops, final_trips.stop_id == close_stops.stop_id , 'inner').select('trip_id').distinct()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
%%spark
final_close_trips1 = final_trips.join(final_close_trips_ids, final_trips.trip_id == final_close_trips_ids.trip_id).drop(final_close_trips_ids.trip_id).distinct()
final_close_trips2 = final_close_trips1.join(all_stops_distance, all_stops_distance.stop_id == final_close_trips1.stop_id).drop(final_close_trips1.stop_id).distinct()
final_close_trips_filtered = final_close_trips2.filter(all_stops_distance['distance'] <= 18000).drop('a','distance', 'lat', 'lon', 'stop_name')
final_close_trips_filtered.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------+-------------+-----------+-------------+----------+------+-------+---------+--------+------+--------------------+------------+
|arrival_time|departure_time|stop_sequence|pickup_type|drop_off_type|service_id|monday|tuesday|wednesday|thursday|friday|             trip_id|     stop_id|
+------------+--------------+-------------+-----------+-------------+----------+------+-------+---------+--------+------+--------------------+------------+
|    10:41:00|      10:41:00|            1|          0|            0|  TA+b0b46|     1|      1|        1|       1|     1|1.TA.26-18-j19-1.1.H|     8503064|
|    10:45:00|      10:45:00|            2|          0|            0|  TA+b0b46|     1|      1|        1|       1|     1|1.TA.26-18-j19-1.1.H| 8503065:0:1|
|    10:46:00|      10:46:00|            3|          0|            0|  TA+b0b46|     1|      1|        1|       1|     1|1.TA.26-18-j19-1.1.H|     8503074|
|    10:47:00|      10:47:00|            4|          0|         

In [27]:
%%spark
final_close_trips_filtered.cache()
final_close_trips_filtered.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

535866

### Making table to link type of mean of transport to trip_ids

In [28]:
%%spark
trips_id = final_close_trips_filtered.select("trip_id").distinct()
trips = spark.read.csv('/data/sbb/csv/trips/2019/05/08/trips.txt', header=True)

trips_filtered = trips.join(trips_id, "trip_id", "inner")

trips_filtered = trips_filtered.select("trip_id", "route_id")

routes = spark.read.csv('/data/sbb/csv/routes/2019/05/08/routes.txt', header=True)
routes_descs = routes.select("route_desc", "route_id")

joined_routes = routes_descs.join(trips_filtered, "route_id", "inner")

joined_routes = joined_routes.select("trip_id", "route_desc", "route_id")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
%%spark
if overwrite:
    joined_routes.write.mode('overwrite').option("delimiter", ";").option("header","true").format("csv").save("/group/five-guys/routes")
else: 
    print("No overwrite, flag set to false")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

No overwrite, flag set to false

## Import SBB

### Import data

In [25]:
%%spark
actual_data = spark.read.orc('/data/sbb/orc/istdaten')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Rename columns

Columns are renamed in English as follows:
- `betriebstag` to DATE
- `produkt_id` to PRODUCT_ID
- `fahrt_bezeichner` to TRIP_ID
- `bpuic` to STOP_ID
- `haltestellen_name` to STOP_NAME
- `ankunftszeit` to ARRIVAL_TIME
- `an_prognose_status` to ARRIVAL_TIME_STATUS
- `an_prognose` to ACTUAL_ARRIVAL_TIME
- `abfahrtszeit` to DEPARTURE_TIME
- `ab_prognose_status` to DEPARTURE_TIME_STATUS
- `ab_prognose` to ACTUAL_DEPARTURE_TIME

In [26]:
%%spark

#rename columns in english
actual_data = actual_data.select(actual_data.betriebstag.alias('DATE'),\
                                 actual_data.produkt_id.alias('PRODUCT_ID'),\
                                 actual_data.bpuic.alias('STOP_ID'),\
                                 actual_data.haltestellen_name.alias('STOP_NAME'),\
                                 actual_data.fahrt_bezeichner.alias('TRIP_ID'),\
                                 actual_data.ankunftszeit.alias('ARRIVAL_TIME'),\
                                 actual_data.an_prognose_status.alias('ARRIVAL_TIME_STATUS'),\
                                 actual_data.an_prognose.alias('ACTUAL_ARRIVAL_TIME'),\
                                 actual_data.abfahrtszeit.alias('DEPARTURE_TIME'),\
                                 actual_data.ab_prognose_status.alias('DEPARTURE_TIME_STATUS'),
                                 actual_data.ab_prognose.alias('ACTUAL_DEPARTURE_TIME'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Prepare table

In [27]:
%%spark
import pyspark.sql.functions as F

df = actual_data.withColumn('DATE', F.date_format(F.to_timestamp(actual_data.DATE, 'dd.MM.yyyy'), 'yyyy-MM-dd'))
df = df.withColumn('DAY_OF_WEEK', ((F.dayofweek(df['DATE'])+5)%7)+1)

#remove entries with ARRIVAL_TIME_STATUS and DEPARTURE_TIME_STATUS unknown
df = df.filter(df['ARRIVAL_TIME_STATUS'] != 'UNBEKANNT')
df = df.filter(df['DEPARTURE_TIME_STATUS'] != 'UNBEKANNT')

#remove entries with ARRIVAL_TIME_STATUS and DEPARTURE_TIME_STATUS is empty
df = df.filter(df['ARRIVAL_TIME_STATUS'] != '')
df = df.filter(df['DEPARTURE_TIME_STATUS'] != '')

#remove entries with PRODUCT_ID empty
df = df.filter(df['PRODUCT_ID'] != '')

#remove entries with STOP_ID empty
df = df.filter(df['STOP_ID'] != '')

#remove entries with STOP_NAME empty
df = df.filter(df['STOP_NAME'] != '')

#remove entries with TRIP_ID empty
df = df.filter(df['TRIP_ID'] != '')

#transform to unix_timestamp
df = df.withColumn('ARRIVAL_TIME', F.unix_timestamp(df.ARRIVAL_TIME, 'dd.MM.yyyy HH:mm'))
df = df.withColumn('ACTUAL_ARRIVAL_TIME', F.unix_timestamp(df.ACTUAL_ARRIVAL_TIME, 'dd.MM.yyyy HH:mm'))
df = df.withColumn('DEPARTURE_TIME', F.unix_timestamp(df.DEPARTURE_TIME, 'dd.MM.yyyy HH:mm'))
df = df.withColumn('ACTUAL_DEPARTURE_TIME', F.unix_timestamp(df.ACTUAL_DEPARTURE_TIME, 'dd.MM.yyyy HH:mm'))

#fill null with 0 
#When the stop is the start or end of a journey, the corresponding columns will be empty (ANKUNFTSZEIT/ABFAHRTSZEIT)
df = df.na.fill({'ARRIVAL_TIME' : 0, 'ACTUAL_ARRIVAL_TIME' : 0, 'DEPARTURE_TIME' : 0, 'ACTUAL_DEPARTURE_TIME' : 0})

#compute delay
df = df.withColumn('ARRIVAL_DELAY', F.when((df['ACTUAL_ARRIVAL_TIME']-df['ARRIVAL_TIME'] )< 0, 0).otherwise(df['ACTUAL_ARRIVAL_TIME']-df['ARRIVAL_TIME']))
df = df.withColumn('DEPARTURE_DELAY', F.when((df['ACTUAL_DEPARTURE_TIME']-df['DEPARTURE_TIME'] )< 0, 0).otherwise(df['ACTUAL_DEPARTURE_TIME']-df['DEPARTURE_TIME']))

#convert seconds in minutes
df = df.withColumn('ARRIVAL_DELAY', F.round(df['ARRIVAL_DELAY']/60))
df = df.withColumn('DEPARTURE_DELAY', F.round(df['DEPARTURE_DELAY']/60))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Filter data based on simplifying assumptions

We only consider journeys at reasonable hours of the day, and on a typical business day.
- `day`: from Monday to Friday
- `hours`: first departure at 9:00, last arrival at 17:59

In [28]:
%%spark

#business day
df = df.filter(df['DAY_OF_WEEK'] <6)
#reasonable hours of the day
df = df.filter(F.hour(F.from_unixtime(df['DEPARTURE_TIME']))>=9)
df = df.filter(F.hour(F.from_unixtime(df['ARRIVAL_TIME']))<18)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
%%spark

final_actual_data = df
final_actual_data.cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[DATE: string, PRODUCT_ID: string, STOP_ID: string, STOP_NAME: string, TRIP_ID: string, ARRIVAL_TIME: bigint, ARRIVAL_TIME_STATUS: string, ACTUAL_ARRIVAL_TIME: bigint, DEPARTURE_TIME: bigint, DEPARTURE_TIME_STATUS: string, ACTUAL_DEPARTURE_TIME: bigint, DAY_OF_WEEK: int, ARRIVAL_DELAY: double, DEPARTURE_DELAY: double]

## Create timetable

As request by the paper 

### Stops Table

In [30]:
%%spark
stops_table = final_close_trips_filtered.select(final_close_trips_filtered.stop_id).distinct()
stops_table.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
%%spark

#keep only stop_ids that are contained in SBB

#make stop_ids compatible with the format of SBB -> remove Parent and P from stop_id
stops_table = stops_table.withColumn('stop_id', F.regexp_replace('stop_id', 'Parent', ''))
stops_table = stops_table.withColumn('stop_id', F.regexp_replace('stop_id', 'P', ''))
final_close_trips_filtered = final_close_trips_filtered.withColumn('stop_id', F.regexp_replace('stop_id', 'Parent', ''))
final_close_trips_filtered = final_close_trips_filtered.withColumn('stop_id', F.regexp_replace('stop_id', 'P', ''))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
%%spark

distinct_stops_SBB = final_actual_data.select('STOP_ID').distinct()
stops_table_without_colon = stops_table.withColumn("stop_id_no_colon", F.split(F.col("stop_id"), ":").getItem(0)).distinct()
stops_table_without_colon = stops_table_without_colon.select("stop_id_no_colon")
distinct_stops_SBB.cache()
stops_table_without_colon.cache()

difference = stops_table_without_colon.subtract(distinct_stops_SBB)
difference.cache()

stops_table_without_colon = stops_table_without_colon.subtract(difference)
stops_table = stops_table.withColumn("stop_id_no_colon", F.split(F.col("stop_id"), ":").getItem(0)).distinct()
stops_table = stops_table.join(stops_table_without_colon, "stop_id_no_colon")
stops_table = stops_table.select('stop_id')
stops_table.cache()
final_close_trips_filtered = final_close_trips_filtered.join(stops_table,'stop_id')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
%%spark

all_stops = spark.read.csv('/data/sbb/csv/allstops/stop_locations.csv')

all_stops = all_stops.withColumnRenamed('_c0','stop_id')
all_stops = all_stops.withColumnRenamed('_c1','stop_name')
all_stops = all_stops.withColumnRenamed('_c2','stop_lat')
all_stops = all_stops.withColumnRenamed('_c3','stop_lon')
all_stops = all_stops.withColumnRenamed('_c4','location_type')
all_stops = all_stops.withColumnRenamed('_c5','parent_station')
all_stops = all_stops.drop('location_type')

all_stops_full = all_stops.join(stops_table, "stop_id" ,"inner")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [203]:
%%spark
all_stops_full.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [213]:
%%spark
if overwrite:
    all_stops_full.write.mode('overwrite').option("delimiter", ";").option("header","true").format("csv").save("/group/five-guys/stops_table")
else: 
    print("No overwrite, flag set to false")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Connections Table

In [34]:
%%spark
trip_ids_with_days_table = final_close_trips_filtered.select('trip_id', 'monday', 'tuesday', 'wednesday', 'thursday', 'friday').distinct()
trip_ids_with_days_table.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+-------+---------+--------+------+
|             trip_id|monday|tuesday|wednesday|thursday|friday|
+--------------------+------+-------+---------+--------+------+
|1.TA.26-18-j19-1.1.H|     1|      1|        1|       1|     1|
|1.TA.63-138-Y-j19...|     0|      0|        1|       0|     0|
|1001.TA.26-213-j1...|     0|      0|        0|       0|     1|
|1014.TA.26-520-j1...|     1|      1|        1|       1|     0|
|1017.TA.26-151-j1...|     1|      1|        1|       1|     0|
|1023.TA.26-4-j19-...|     1|      1|        1|       1|     1|
|1029.TA.26-LAF-j1...|     1|      1|        1|       1|     0|
|103.TA.26-510-j19...|     0|      0|        0|       0|     1|
|1039.TA.26-200-j1...|     0|      0|        0|       0|     1|
|1082.TA.26-201-j1...|     0|      0|        0|       0|     1|
+--------------------+------+-------+---------+--------+------+
only showing top 10 rows

In [35]:
%%spark
working_table = final_close_trips_filtered.withColumn('stop_sequence',final_close_trips_filtered.stop_sequence.cast('int')).orderBy('trip_id', 'stop_sequence').drop('service_id', 'pickup_type', 'drop_off_type', 'monday',  'tuesday', 'thursday', 'wednesday', 'thursday', 'friday')
working_table = working_table.withColumnRenamed('stop_id','departure_id')
working_table = working_table.withColumnRenamed('arrival_time','old_arrival_time')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
%%spark
import pyspark.sql.functions as F
from pyspark.sql.window import Window

w = Window().partitionBy('trip_id').orderBy('stop_sequence')
table_a = working_table.select("*",F.lag("old_arrival_time", -1).over(w).alias("arrival_time"), F.lag("departure_id", -1).over(w).alias("arrival_id")).na.drop()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
%%spark
import pyspark.sql.functions as F
from pyspark.sql.window import Window

final_table = table_a.join(trip_ids_with_days_table, table_a.trip_id == trip_ids_with_days_table.trip_id)
connections_table = final_table.drop('old_arrival_time', 'stop_sequence').select('departure_id', 'arrival_id', 'departure_time', 'arrival_time', table_a.trip_id, 'monday', 'tuesday', 'wednesday', 'thursday', 'friday')
connections_table.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-----------+--------------+------------+--------------------+------+-------+---------+--------+------+
|departure_id| arrival_id|departure_time|arrival_time|             trip_id|monday|tuesday|wednesday|thursday|friday|
+------------+-----------+--------------+------------+--------------------+------+-------+---------+--------+------+
|     8503082|    8503081|      13:20:00|    13:25:00|1029.TA.26-LAF-j1...|     1|      1|        1|       1|     0|
|     8573257|8503702:0:E|      10:19:00|    10:21:00|190.TA.26-530-j19...|     1|      1|        1|       1|     0|
|     8503850|    8573257|      10:17:00|    10:19:00|190.TA.26-530-j19...|     1|      1|        1|       1|     0|
|     8503772|    8503850|      10:16:00|    10:17:00|190.TA.26-530-j19...|     1|      1|        1|       1|     0|
|     8582739|    8503772|      10:15:00|    10:16:00|190.TA.26-530-j19...|     1|      1|        1|       1|     0|
|     8503781|    8582739|      10:13:00|    10:15:00|190.TA.26-

#### Check correctness of the window join, it should be 0

In [38]:
%%spark
connections_table.filter(connections_table.arrival_time < connections_table.departure_time).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

In [39]:
%%spark
connections_table.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

495470

In [40]:
%%spark
from pyspark.sql.functions import desc
#ordering of columns
connections_table = connections_table.orderBy(desc("departure_time"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
%%spark
if overwrite:
    connections_table.write.mode('overwrite').option("header","true").format("csv").save("/group/five-guys/conn_table")
else: 
    print("No overwrite, flag set to false")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Trips Table

In [42]:
%%spark
#final_close_trips_filtered = final_close_trips.filter(final_close_trips.arrival_time>final_close_trips.departure_time)
trips_table = final_close_trips_filtered.select(final_close_trips_filtered.trip_id).distinct()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
%%spark 
#trips as described in the readme, but filtered for time and distance

trips_table_filtered = routes_tot.join(trips_table, "trip_id" ,"inner")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
%%spark
trips_table_filtered = trips_table_filtered.select("route_id", "service_id", "trip_id", "trip_headsign", "trip_short_name", "direction_id")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
%%spark

if overwrite:
    trips_table_filtered.write.mode('overwrite').option("delimiter", ";").option("header","true").format("csv").save("/group/five-guys/trips")
else: 
    print("No overwrite, flag set to false")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

No overwrite, flag set to false

### Footpath table

In [46]:
%%spark

#add attributes lat, lon to stops_table
stops_table_expanded = stops_table.join(all_stops, 'stop_id', 'inner')
#create all possible pairs between stops
stops_table_expanded_a = stops_table_expanded.select([F.col(c).alias("a_"+c) for c in stops_table_expanded.columns])
stops_table_expanded_b = stops_table_expanded.select([F.col(c).alias("b_"+c) for c in stops_table_expanded.columns])
cartesian_product = stops_table_expanded_a.crossJoin(stops_table_expanded_b)
#compute distance for all pairs
stops_table_distance = cartesian_product.withColumn("a", (
        F.pow(F.sin(F.radians(F.col("a_stop_lat") - F.col("b_stop_lat")) / 2), 2) +
        F.cos(F.radians(F.col("b_stop_lat"))) * F.cos(F.radians(F.col("a_stop_lat"))) *
        F.pow(F.sin(F.radians(F.col("a_stop_lon") - F.col("b_stop_lon")) / 2), 2)
    )).withColumn("distance", F.atan2(F.sqrt(F.col("a")), F.sqrt(-F.col("a") + 1)) * 12742000)
#filter pair with distance shorter than 500m
stops_table_distance = stops_table_distance.filter(stops_table_distance.distance <= 500)
#compute time considering walking speed of 50m/1min
#time is in seconds
stops_table_time = stops_table_distance.withColumn('time', F.round((stops_table_distance['distance']/F.lit(50))*F.lit(60), 0).cast('int'))
footpath_table = stops_table_time.select(stops_table_time.a_stop_id.alias('departure_id'), stops_table_time.b_stop_id.alias('arrival_id'), stops_table_time.time)
#remove pairs where arrival == departure
footpath_table = footpath_table.filter(footpath_table.departure_id != footpath_table.arrival_id)
footpath_table.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-----------+----+
|departure_id| arrival_id|time|
+------------+-----------+----+
| 8503306:0:2|    8590541| 528|
|     8506895|    8573228| 402|
|     8591190|    8591170| 311|
|     8591284|    8591116| 479|
|     8591284|    8591274| 327|
|     8503078|    8591903| 471|
|     8589111|    8590655| 583|
|     8589111|    8594182| 293|
| 8503306:0:2|8503306:0:1|   0|
|     8589111|    8502208| 351|
+------------+-----------+----+
only showing top 10 rows

In [47]:
%%spark

if overwrite:
    footpath_table.write.mode('overwrite').option("delimiter", ";").option("header","true").format("csv").save("/group/five-guys/footpath")
else: 
    print("No overwrite, flag set to false")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

No overwrite, flag set to false

## Confidence computation

In [39]:
%%spark
#remove stops from SBB that are not in stops_table

stops_table_with_no_colon = stops_table.withColumn("STOP_ID", F.split(F.col("stop_id"), ":").getItem(0)).distinct()
stops_table_with_no_colon = stops_table_with_no_colon.select("STOP_ID")
final_actual_data = final_actual_data.join(stops_table_with_no_colon,'STOP_ID')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
%%spark

#for each entry in SBB define DEPARTURE_ID and ARRIVAL_ID
w = Window().partitionBy('TRIP_ID', 'DATE').orderBy('ARRIVAL_TIME')
df_a = df.select("*",F.lag("STOP_ID", +1).over(w).alias("DEPARTURE_ID")).na.drop()
df_a = df_a.withColumnRenamed('STOP_ID','ARRIVAL_ID')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
%%spark
df_2 = df_a.select(df_a.DEPARTURE_ID, df_a.ARRIVAL_ID, df_a.ARRIVAL_DELAY, df_a.DAY_OF_WEEK)
df_2.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Obtain delay cumulative distribution

In [43]:
%%spark
from pyspark.sql import Window

#compute cumulative distribution of delay over groupBy 'DAY_OF_WEEK', 'PRODUCT_ID', 'HOUR_ARRIVAL' 
cumulative_function_window = Window.partitionBy('DAY_OF_WEEK', 'DEPARTURE_ID', 'ARRIVAL_ID').orderBy('ARRIVAL_DELAY').rangeBetween(Window.unboundedPreceding, 0)
count_window = Window.partitionBy('DAY_OF_WEEK', 'DEPARTURE_ID', 'ARRIVAL_ID')

df2 = df_2.groupBy(df_2['DAY_OF_WEEK'], df_2['DEPARTURE_ID'], df_2['ARRIVAL_ID'], df_2['ARRIVAL_DELAY']).count()
df2 = df2.withColumn("TOTAL_SUM", F.sum("count").over(count_window))
df2 = df2.withColumn("PARTIAL_SUM", F.sum("count").over(cumulative_function_window))
df2 = df2.withColumn("CUMULATIVE", F.sum("count").over(cumulative_function_window)/F.sum("count").over(count_window))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
%%spark

#reorder columns
df2 = df2.select(df2["DEPARTURE_ID"], df2["ARRIVAL_ID"], df2["DAY_OF_WEEK"], df2["ARRIVAL_DELAY"].alias("MAX_ARRIVAL_DELAY"), df2["CUMULATIVE"])
df2.cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Compute mean and variance for the missing stations in the algorithm

In [233]:
%%spark

#compute mean value
df2.select(F.mean('CUMULATIVE')).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<bound method DataFrame.cache of DataFrame[DEPARTURE_ID: string, ARRIVAL_ID: string, DAY_OF_WEEK: int, MAX_ARRIVAL_DELAY: double, CUMULATIVE: double]>

In [234]:
%%spark 

#compute std deviation
df2.select(F.stddev_samp('CUMULATIVE')).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [235]:
%%spark
df2.orderBy('DEPARTURE_ID','ARRIVAL_ID', 'DAY_OF_WEEK', 'MAX_ARRIVAL_DELAY').show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
%%spark
#write table on hdfs
if overwrite:
    df2.write.mode('overwrite').option("delimiter", ";").option("header","true").format("csv").save("/group/five-guys/confidence")
else: 
    print("No overwrite, flag set to false")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…